In [ ]:
import pandas as pd

data_rate = pd.read_csv('制造业清洗完成的数据.csv')

data_X = data_rate[~data_rate['FLAG'].isnull()]
data_Y = data_rate[data_rate['FLAG'].isnull()]
data_X['FLAG'] = data_X['FLAG'].apply(lambda x :int(x))

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE   # SMOTE采样
from collections import Counter
from sklearn.ensemble import AdaBoostClassifier

for i in range(100):
    X_0 = data_X[data_X['FLAG']==0].sample(len(data_X[data_X['FLAG']==1])*3)
    X_1 = data_X[data_X['FLAG']==1]
    X_0 = X_0.append(X_1)
    
    X_tr, X_te, Y_tr, Y_te = train_test_split(X_0.iloc[:,12:-1],X_0.iloc[:,-1],test_size=0.25)   # 拆分训练集、测试集,12,134

    sos = SMOTE(random_state=0)  # SMOTE采样
    X_sos, y_sos = sos.fit_resample(X_tr, Y_tr)

    clf = AdaBoostClassifier(n_estimators=1000, random_state=2,learning_rate=0.25)
    clf.fit(X_sos, y_sos)

    pre = clf.predict(X_te)
    DataF[str(i)] = pre
    acc_te = sum(pre == Y_te)/len(pre)
    print('测试集的准确率',acc_te)
    
    indices = np.argsort(clf.feature_importances_)[::-1]
    f = open('制造业指标.csv',mode='w')
    for i in X_0.columns.values[12:][indices][:20]:
        f.write(i)
        f.write(',')
    f.write('\n')
    f.close()